In [14]:
import pandas as pd 

stock_data=pd.read_html("https://www.finanznachrichten.de/nachrichten/alle-empfehlungen.html")



In [16]:

len(stock_data)

4

In [18]:
stock_data_first=pd.DataFrame(stock_data[1])
print(stock_data_first.head())
stock_data_first.columns

    Zeit                           Aktuelle Nachrichten  \
0  21:46  JPMORGAN belässt COMMERZBANK AG auf 'Neutral'   
1  21:46            GOLDMAN SACHS belässt ENI auf 'Buy'   
2  19:10         BAADER BANK stuft Puma SE auf 'Reduce'   
3  17:22       UBS belässt Siemens Gamesa auf 'Neutral'   
4  16:22            UBS belässt Gea Group auf 'Neutral'   

                 Unternehmen / Aktien        PotentialKurse aktuell  Leser  \
0                      COMMERZBANK AG       -9 % (5 Euro)5,496 Euro  305.0   
1                             ENI SPA  +26 % (10,50 Euro)8,352 Euro  274.0   
2                             PUMA SE      -4 % (80 Euro)83,56 Euro  447.0   
3  SIEMENS GAMESA RENEWABLE ENERGY SA    -19 % (30 Euro)36,920 Euro  488.0   
4                        GEA GROUP AG    +12 % (33 Euro)29,530 Euro  522.0   

   Unnamed: 5  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  


Index(['Zeit', 'Aktuelle Nachrichten', 'Unternehmen / Aktien',
       'PotentialKurse aktuell', 'Leser', 'Unnamed: 5'],
      dtype='object')

In [20]:
import re

stock_data_first['Aktuelle Nachrichten']=stock_data_first['Aktuelle Nachrichten'].str.findall('(?:stuft|belässt)\s([A-Za-z]+)').transform(''.join)
stock_data_first.head()

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5
0,21:46,COMMERZBANK,COMMERZBANK AG,"-9 % (5 Euro)5,496 Euro",305.0,NaN
1,21:46,ENI,ENI SPA,"+26 % (10,50 Euro)8,352 Euro",274.0,NaN
2,19:10,Puma,PUMA SE,"-4 % (80 Euro)83,56 Euro",447.0,NaN
3,17:22,Siemens,SIEMENS GAMESA RENEWABLE ENERGY SA,"-19 % (30 Euro)36,920 Euro",488.0,NaN
4,16:22,Gea,GEA GROUP AG,"+12 % (33 Euro)29,530 Euro",522.0,NaN


In [22]:
stock_data_first['Potential in %']=stock_data_first['PotentialKurse aktuell'].str.findall('([+-]\d*)\s%').transform(''.join).apply(pd.to_numeric)


In [23]:
stock_data_first.head()

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5,Potential in %
0,21:46,COMMERZBANK,COMMERZBANK AG,"-9 % (5 Euro)5,496 Euro",305.0,NaN,-9.0
1,21:46,ENI,ENI SPA,"+26 % (10,50 Euro)8,352 Euro",274.0,NaN,26.0
2,19:10,Puma,PUMA SE,"-4 % (80 Euro)83,56 Euro",447.0,NaN,-4.0
3,17:22,Siemens,SIEMENS GAMESA RENEWABLE ENERGY SA,"-19 % (30 Euro)36,920 Euro",488.0,NaN,-19.0
4,16:22,Gea,GEA GROUP AG,"+12 % (33 Euro)29,530 Euro",522.0,NaN,12.0


In [24]:
stock_data_first=stock_data_first.sort_values(by=['Potential in %'],ascending=False)
stock_data_first.head()

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5,Potential in %
8,14:22,SAINT,COMPAGNIE DE SAINT-GOBAIN SA,"+39 % (59 Euro)42,360 Euro",936.000,NaN,39.0
5,15:22,Airbus,AIRBUS SE,"+28 % (109 Euro)84,84 Euro",1.087,NaN,28.0
1,21:46,ENI,ENI SPA,"+26 % (10,50 Euro)8,352 Euro",274.000,NaN,26.0
9,14:10,ING,ING GROEP NV,"+23 % (9 Euro)7,334 Euro",999.000,NaN,23.0
7,14:46,Covestro,COVESTRO AG,"+21 % (67 Euro)55,36 Euro",869.000,NaN,21.0


In [25]:
from bs4 import BeautifulSoup
import requests

page=requests.get("https://www.finanznachrichten.de/nachrichten/alle-empfehlungen.html")
soup=BeautifulSoup(page.content,"html.parser")

In [26]:
best_five=stock_data_first.iloc[:5]

In [27]:
best_five

,Zeit,Aktuelle Nachrichten,Unternehmen / Aktien,PotentialKurse aktuell,Leser,Unnamed: 5,Potential in %
8,14:22,SAINT,COMPAGNIE DE SAINT-GOBAIN SA,"+39 % (59 Euro)42,360 Euro",936.000,NaN,39.0
5,15:22,Airbus,AIRBUS SE,"+28 % (109 Euro)84,84 Euro",1.087,NaN,28.0
1,21:46,ENI,ENI SPA,"+26 % (10,50 Euro)8,352 Euro",274.000,NaN,26.0
9,14:10,ING,ING GROEP NV,"+23 % (9 Euro)7,334 Euro",999.000,NaN,23.0
7,14:46,Covestro,COVESTRO AG,"+21 % (67 Euro)55,36 Euro",869.000,NaN,21.0


In [28]:
tables_list=[x for x in soup.find_all("table",attrs={"class":"rel-ct"})]

In [29]:
links=tables_list[0].tbody.find_all('a')
filtered_links=[x.get('href') for i,x in enumerate(links) if i%2==0]


In [30]:
best_links=list(map(lambda x:list([(y,x) for y in filtered_links if x in y]),best_five['Aktuelle Nachrichten'].str.lower()))
best_links=list(map(lambda x: x[0],best_links))
best_links

[('https://www.finanznachrichten.de/nachrichten-2021-01/51818939-barclays-belaesst-saint-gobain-auf-overweight-322.htm',
  'saint'),
 ('https://www.finanznachrichten.de/nachrichten-2021-01/51819832-jpmorgan-belaesst-airbus-auf-overweight-322.htm',
  'airbus'),
 ('https://www.finanznachrichten.de/nachrichten-2021-01/51823099-goldman-sachs-belaesst-eni-auf-buy-322.htm',
  'eni'),
 ('https://www.finanznachrichten.de/nachrichten-2021-01/51818753-credit-suisse-belaesst-ing-groep-n-v-auf-outperform-322.htm',
  'ing'),
 ('https://www.finanznachrichten.de/nachrichten-2021-01/51819272-goldman-sachs-belaesst-covestro-auf-buy-322.htm',
  'covestro')]

In [31]:
isins=[]
for li in best_links:
    page_2=requests.get(li[0])
    soup_2=BeautifulSoup(page_2.content,"html.parser")
    span=soup_2.find('span',attrs={'class':"wkn-isin"})
    isins.append((span.find_all(id="produkt-isin")[0].get('data-isin'),*li))

In [32]:
isins

[('FR0000125007',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51818939-barclays-belaesst-saint-gobain-auf-overweight-322.htm',
  'saint'),
 ('NL0000235190',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51819832-jpmorgan-belaesst-airbus-auf-overweight-322.htm',
  'airbus'),
 ('IT0003132476',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51823099-goldman-sachs-belaesst-eni-auf-buy-322.htm',
  'eni'),
 ('NL0011821202',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51818753-credit-suisse-belaesst-ing-groep-n-v-auf-outperform-322.htm',
  'ing'),
 ('DE0006062144',
  'https://www.finanznachrichten.de/nachrichten-2021-01/51819272-goldman-sachs-belaesst-covestro-auf-buy-322.htm',
  'covestro')]

In [33]:
stock_data_onvista=pd.DataFrame([])
for isin in isins:
    #page_3=requests.get("https://www.onvista.de/aktien/fundamental/-Aktie-{}".format(isin[0]))
    data=pd.read_html("https://www.onvista.de/aktien/fundamental/-Aktie-{}".format(isin[0]))
    filtered_data=data[1].rename(columns={data[1].columns[0]:"Stats"})
    filtered_data['Unternehmen']=isin[2]
    filtered_data=filtered_data.set_index(filtered_data['Unternehmen']).iloc[2]
    stock_data_onvista=stock_data_onvista.append(filtered_data) 


In [34]:
stock_data_onvista

,2017,2018,2019,2020e,2021e,2022e,2023e,Stats,Unternehmen
saint,"+19,91%","-72,79%","+236,36%","-38,38%","+112,23%","+17,15%","+5,34%",Gewinnwachstum,saint
airbus,"+136,43%","+29,18%","+47,40%","-11,14%",-,"+28,42%","+41,60%",Gewinnwachstum,airbus
eni,-,"+22,34%","-96,52%",-,-,"+98,69%","+31,63%",Gewinnwachstum,eni
ing,"+5,00%","-3,96%","+1,65%","-52,42%","+49,33%","+19,68%","-0,57%",Gewinnwachstum,ing
covestro,"+152,67%","-4,73%","-68,07%","-18,87%","+24,83%","+35,13%","+52,91%",Gewinnwachstum,covestro
